In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
labels = df[1].values

In [4]:
type(labels)

numpy.ndarray

In [5]:
texts = df[0].values.tolist()

In [6]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

In [7]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

C:\Users\Sang\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sang\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [8]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [10]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [11]:
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_roberta_en"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [14]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, validation_split=0.1, 
         callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 578s 15s/step - loss: 0.5226 - accuracy: 0.7192 - val_loss: 0.3050 - val_accuracy: 0.8845
Epoch 2/10
39/39 [==============================] - 966s 25s/step - loss: 0.2715 - accuracy: 0.8994 - val_loss: 0.2189 - val_accuracy: 0.9206
Epoch 3/10
39/39 [==============================] - 941s 24s/step - loss: 0.1774 - accuracy: 0.9396 - val_loss: 0.2254 - val_accuracy: 0.9188
Epoch 4/10
39/39 [==============================] - 893s 23s/step - loss: 0.1165 - accuracy: 0.9629 - val_loss: 0.2591 - val_accuracy: 0.9116
Epoch 5/10
39/39 [==============================] - 935s 24s/step - loss: 0.0841 - accuracy: 0.9731 - val_loss: 0.2786 - val_accuracy: 0.9116
Epoch 5: early stopping


In [16]:
model.load_weights(checkpoint_filepath)

In [17]:
model.evaluate(dict(X_test_tokenized), np.array(y_test))

44/44 [==============================] - 96s 2s/step - loss: 0.2195 - accuracy: 0.9169


[0.21948546171188354, 0.9169074892997742]

In [18]:
y_preds = model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

44/44 [==============================] - 109s 2s/step
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       643
           1       0.92      0.92      0.92       741

    accuracy                           0.92      1384
   macro avg       0.92      0.92      0.92      1384
weighted avg       0.92      0.92      0.92      1384

